In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

In [3]:
import pandas as pd

In [4]:
from d00_utils.db_utils import dbReadWriteClean, dbReadWriteViews, dbReadWriteSegmentation
io_clean = dbReadWriteClean()
io_views = dbReadWriteViews()
io_segmentation = dbReadWriteSegmentation()

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
# PARAM
instances_w_labels_test_downsampleby5_df = io_views.get_table('instances_w_labels_test_downsampleby5')
instances_w_labels_test_downsampleby5_df = instances_w_labels_test_downsampleby5_df[(instances_w_labels_test_downsampleby5_df['view']=='a2c') | (instances_w_labels_test_downsampleby5_df['view']=='a4c')]
instances_w_labels_test_downsampleby5_df = instances_w_labels_test_downsampleby5_df[['studyidk', 'instanceidk', 'filename']]
instances = instances_w_labels_test_downsampleby5_df['instanceidk'].unique()
print(len(instances))

522


In [6]:
def get_recommendation(row):
    return 'normal' if row['ef'] >= 60 else 'abnormal' if row['ef'] < 40 else 'greyzone'

In [7]:
measurement_abstract_rpt_df = io_clean.get_table("measurement_abstract_rpt")
measurement_abstract_rpt_df = measurement_abstract_rpt_df.drop(['value'], axis=1)

a_measgraphref_df = io_clean.get_table("a_measgraphref")
a_measgraphref_df = a_measgraphref_df.drop(['srinstanceidk', 'imagesopinstanceuid', 'measurementuid'], axis=1)

instances_w_labels_df = io_views.get_table('instances_w_labels')
instances_w_labels_df = instances_w_labels_df[(instances_w_labels_df['view']!='plax')]
instances_w_labels_df = instances_w_labels_df[['studyidk', 'instanceidk', 'filename']]

/home/ubuntu/anaconda3/envs/usal_echo/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
merge_df = measurement_abstract_rpt_df.merge(a_measgraphref_df, on=['studyidk', 'measabstractnumber'])
merge_df = merge_df.merge(instances_w_labels_df, on=['studyidk', 'instanceidk'])

In [9]:
filter_df = merge_df[merge_df['name'].isin(['VTD(MDD-ps4)', 'VTS(MDD-ps4)'])]
filter_df = filter_df[filter_df['value']>0]
filter_df = filter_df[filter_df['instanceidk'].isin(instances)]

In [10]:
volume_df = filter_df[['studyidk', 'instanceidk', 'filename', 'name', 'unitname', 'value', 'indexinmglist']]
volume_df = volume_df.rename(
    columns={'studyidk': 'study_id', 'instanceidk': 'instance_id', 'filename': 'file_name', 'name': 'measurement_name', 'value': 'measurement_value', 'unitname': 'measurement_unit'})
print(volume_df.shape)
volume_df.head()

(472, 7)


,study_id,instance_id,file_name,measurement_name,measurement_unit,measurement_value,indexinmglist
247,43763,779540,1LHFYKEZ,VTD(MDD-ps4),mL,45.964351,0
251,43763,779540,1LHFYKEZ,VTS(MDD-ps4),mL,17.993407,1
471,46919,841585,7SQZPLEJ,VTD(MDD-ps4),mL,124.449981,0
475,46919,841585,7SQZPLEJ,VTS(MDD-ps4),mL,60.740148,1
655,50521,921901,EAKXA40T,VTD(MDD-ps4),mL,91.569690,0


In [11]:
agg_dict = {"measurement_value": pd.Series.median, "measurement_unit": pd.Series.unique}
volume_df = volume_df.groupby(['study_id', 'instance_id', 'file_name', 'measurement_name', 'indexinmglist']).agg(agg_dict).reset_index()
volume_df = volume_df.groupby(['study_id', 'instance_id', 'file_name', 'measurement_name']).agg(agg_dict).reset_index()
print(volume_df.shape)
volume_df.head()

(471, 6)


,study_id,instance_id,file_name,measurement_name,measurement_value,measurement_unit
0,43763,779540,1LHFYKEZ,VTD(MDD-ps4),45.964351,mL
1,43763,779540,1LHFYKEZ,VTS(MDD-ps4),17.993407,mL
2,46919,841585,7SQZPLEJ,VTD(MDD-ps4),124.449981,mL
3,46919,841585,7SQZPLEJ,VTS(MDD-ps4),60.740148,mL
4,50521,921901,EAKXA40T,VTD(MDD-ps4),91.569690,mL


In [12]:
diastole_df = volume_df[volume_df['measurement_name'].str.contains('VTD')]
systole_df = volume_df[volume_df['measurement_name'].str.contains('VTS')]
print(diastole_df.shape)

(238, 6)


In [13]:
diastole_df = diastole_df[diastole_df['instance_id'].isin(systole_df['instance_id'].unique())]
systole_df = systole_df[systole_df['instance_id'].isin(diastole_df['instance_id'].unique())]
print(diastole_df.shape)

(233, 6)


In [14]:
diastole_df = diastole_df.drop(['measurement_name', 'measurement_unit'], axis=1)
systole_df = systole_df.drop(['measurement_name', 'measurement_unit'], axis=1)

In [15]:
diastole_df.head()

,study_id,instance_id,file_name,measurement_value
0,43763,779540,1LHFYKEZ,45.964351
2,46919,841585,7SQZPLEJ,124.449981
4,50521,921901,EAKXA40T,91.569690
6,57515,1078635,AFUJWQM9,73.781595
8,60906,1168238,FONQVY2N,132.222893


In [16]:
ef_df = diastole_df.merge(systole_df, on=['study_id', 'instance_id'], suffixes=['_diastole', '_systole'])
ef_df = ef_df[ef_df['measurement_value_diastole']>=ef_df['measurement_value_systole']]
print(ef_df.shape)
ef_df.head()

(233, 6)


,study_id,instance_id,file_name_diastole,measurement_value_diastole,file_name_systole,measurement_value_systole
0,43763,779540,1LHFYKEZ,45.964351,1LHFYKEZ,17.993407
1,46919,841585,7SQZPLEJ,124.449981,7SQZPLEJ,60.740148
2,50521,921901,EAKXA40T,91.569690,EAKXA40T,40.583427
3,57515,1078635,AFUJWQM9,73.781595,AFUJWQM9,31.494208
4,60906,1168238,FONQVY2N,132.222893,FONQVY2N,57.683639


In [17]:
ef_df['file_name'] = ef_df['file_name_diastole']
ef_df['measurement_name'] = 'FE(MDD-ps4)'
ef_df['measurement_value'] = (ef_df['measurement_value_diastole']-ef_df['measurement_value_systole'])/ef_df['measurement_value_diastole']*100
ef_df['measurement_unit'] = '%'
ef_df = ef_df.drop(['file_name_diastole', 'measurement_value_diastole', 'file_name_systole', 'measurement_value_systole'], axis=1)
ef_df.head()

,study_id,instance_id,file_name,measurement_name,measurement_value,measurement_unit
0,43763,779540,1LHFYKEZ,FE(MDD-ps4),60.853560,%
1,46919,841585,7SQZPLEJ,FE(MDD-ps4),51.193124,%
2,50521,921901,EAKXA40T,FE(MDD-ps4),55.680283,%
3,57515,1078635,AFUJWQM9,FE(MDD-ps4),57.314276,%
4,60906,1168238,FONQVY2N,FE(MDD-ps4),56.373940,%


In [18]:
volume_df.head()

,study_id,instance_id,file_name,measurement_name,measurement_value,measurement_unit
0,43763,779540,1LHFYKEZ,VTD(MDD-ps4),45.964351,mL
1,43763,779540,1LHFYKEZ,VTS(MDD-ps4),17.993407,mL
2,46919,841585,7SQZPLEJ,VTD(MDD-ps4),124.449981,mL
3,46919,841585,7SQZPLEJ,VTS(MDD-ps4),60.740148,mL
4,50521,921901,EAKXA40T,VTD(MDD-ps4),91.569690,mL


In [19]:
ef_df.head()

,study_id,instance_id,file_name,measurement_name,measurement_value,measurement_unit
0,43763,779540,1LHFYKEZ,FE(MDD-ps4),60.853560,%
1,46919,841585,7SQZPLEJ,FE(MDD-ps4),51.193124,%
2,50521,921901,EAKXA40T,FE(MDD-ps4),55.680283,%
3,57515,1078635,AFUJWQM9,FE(MDD-ps4),57.314276,%
4,60906,1168238,FONQVY2N,FE(MDD-ps4),56.373940,%


In [20]:
ground_truths = volume_df.append(ef_df)

In [21]:
io_segmentation.save_to_db(ground_truths, "ground_truths")